# Visualization Variant Painting Images and Cells

In [22]:
import polars as pl
import os
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage.io import imread
from tqdm import tqdm
import re

CP_GALLERY_DIR = "../../../../2_analysis/3_cytoself/cellpainting_gallery"
batches = ["2024_01_23_Batch_7", "2024_02_06_Batch_8"]

## Loading meta data and variant classification

### Read in the meta data

In [23]:
meta_data = pl.DataFrame()
plate_map_dirs = [
    f"../snakemake_pipeline/inputs/metadata/platemaps/{batch_id}/platemap/" 
    for batch_id in batches
]
for plate_map_dir in plate_map_dirs:
    for plate_map in os.listdir(plate_map_dir):
        if plate_map.endswith(".txt") and plate_map.startswith("B7A1R1_P"):
            plate_map_path = os.path.join(plate_map_dir, plate_map)
            plate_map_data = pl.read_csv(plate_map_path, separator="\t", infer_schema_length=1000000)
            # print(plate_map_data)
            if meta_data.is_empty():
                meta_data = plate_map_data
            else:
                meta_data = pl.concat([meta_data, plate_map_data], how="vertical_relaxed")

display(meta_data.head())
display(meta_data.tail())

plate_map_name,well_position,control_type,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter
str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,i64,str
"""B7A1R1_P1""","""A01""","""disease_wt""","""ACSF3""","""ACSF3""","""A01""","""B7A1R1_P1""","""B8A1R2_P1""","""disease_wt""",71337.0,null,null,null,1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,1,1,1,"""A"""
"""B7A1R1_P1""","""C01""","""allele""","""ACSF3""","""ACSF3_Ala197Thr""","""C01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,3943.0,"""589G>A""","""Ala197Thr""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,2,3,1,"""C"""
"""B7A1R1_P1""","""E01""","""allele""","""ACSF3""","""ACSF3_Thr358Ile""","""E01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,205771.0,"""1073C>T""","""Thr358Ile""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,3,5,1,"""E"""
"""B7A1R1_P1""","""G01""","""allele""","""ACSF3""","""ACSF3_Met198Arg""","""G01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,3944.0,"""593T>G""","""Met198Arg""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,4,7,1,"""G"""
"""B7A1R1_P1""","""I01""","""allele""","""ACSF3""","""ACSF3_Glu359Lys""","""I01""","""B7A1R1_P1""","""B8A1R2_P1""","""allele""",71337.0,205770.0,"""1075G>A""","""Glu359Lys""",1.0,1,7,8,null,null,null,null,527.0,null,1,1,1,5,9,1,"""I"""


plate_map_name,well_position,control_type,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,hgmd_ID,clinvar_allele_ID,clinvar_clinical_significance,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter
str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,str,str,str,str,f64,str,i64,i64,i64,i64,i64,i64,str
"""B7A1R1_P2""","""H24""","""disease_wt""","""MAP2K2""","""MAP2K2""","""H24""","""B7A1R1_P2""","""B8A1R2_P2""","""disease_wt""",6666.0,null,null,null,1.0,2,7,8,null,null,null,null,527.0,null,2,4,12,4,8,24,"""H"""
"""B7A1R1_P2""","""J24""","""allele""","""MAP2K2""","""MAP2K2_Thr17Ala""","""J24""","""B7A1R1_P2""","""B8A1R2_P2""","""allele""",6666.0,51.0,"""49A>G""","""Thr17Ala""",1.0,2,7,8,null,null,null,null,527.0,null,2,4,12,5,10,24,"""J"""
"""B7A1R1_P2""","""L24""","""disease_wt""","""MAX""","""MAX""","""L24""","""B7A1R1_P2""","""B8A1R2_P2""","""disease_wt""",4384.0,null,null,null,1.0,2,7,8,null,null,null,null,527.0,null,2,4,12,6,12,24,"""L"""
"""B7A1R1_P2""","""N24""","""disease_wt""","""MCEE""","""MCEE""","""N24""","""B7A1R1_P2""","""B8A1R2_P2""","""disease_wt""",13113.0,null,null,null,1.0,2,7,8,null,null,null,null,527.0,null,2,4,12,7,14,24,"""N"""
"""B7A1R1_P2""","""P24""","""allele""","""NAA10""","""NAA10_Arg195Cys""","""P24""","""B7A1R1_P2""","""B8A1R2_P2""","""allele""",3180.0,243.0,"""583C>T""","""Arg195Cys""",1.0,2,7,8,null,null,null,null,527.0,null,2,4,12,8,16,24,"""P"""


### Classification of variant

In [3]:
# trn_imbalance_thresh = 3

# Paths
metrics_dir = "../snakemake_pipeline/outputs/analyses/{}/profiles_tcdropped_filtered_var_mad_outlier_featselect"
metrics_df, metrics_wtvar = pl.DataFrame(), pl.DataFrame()
for batch in batches:
    met_dir = metrics_dir.format(batch)
    metrics_df_batch = pl.read_csv(f"{met_dir}/metrics.csv")
    metrics_wtvar_batch = pl.read_csv(f"{met_dir}/metrics_summary.csv")

    metrics_df = pl.concat([metrics_df, metrics_df_batch])
    metrics_wtvar = pl.concat([metrics_wtvar, metrics_wtvar_batch])

## get the localization metrics
local_wtvar = metrics_wtvar.filter(pl.col("Classifier_type") == "localization")
## get the alleles and their reference genes
local_wtvar = local_wtvar.with_columns(
    pl.when(pl.col("AUROC_mean") > pl.col("AUROC_thresh"))
    .then(1)
    .otherwise(0)
    .alias("mislocalized")
).unique()

## get the localization metrics
morph_wtvar = metrics_wtvar.filter(pl.col("Classifier_type") == "morphology")
## get the alleles and their reference genes
morph_wtvar = morph_wtvar.with_columns(
    pl.when(pl.col("AUROC_mean") > pl.col("AUROC_thresh"))
    .then(1)
    .otherwise(0)
    .alias("morph_change")
).unique()

display(morph_wtvar.head())
display(local_wtvar.head())

# misloc_alleles = (
#     local_wtvar
#     .select("allele_0")
#     .to_series()
#     .to_list()
# )
# misloc_binary = (
#     local_wtvar.pivot(
#         index=["allele_0", "Allele_set"], ## , "AUROC_mean"
#         columns="Batch",
#         values="AUROC_mean",
#     )
#     .rename({"B7": "AUROC_mean_B7", "B8": "AUROC_mean_B8"})
# )
# misloc_binary

# misloc_alleles = (
#     morph_wtvar
#     .select("allele_0")
#     .to_series()
#     .to_list()
# )
# morph_binary = (
#     morph_wtvar.pivot(
#         index=["allele_0", "Allele_set"], ## , "AUROC_mean"
#         columns="Batch",
#         values=["morph_change"],
#     )
#     .with_columns(
#         ((pl.col("B7") == 1) & (pl.col("B8") == 1)).alias(
#             "morph_change_both_batches"
#         )
#     )
#     .rename({"B7": "morph_change_batch7", "B8": "morph_change_batch8"})
#     .filter(pl.col("allele_0").is_in(ind_class["allele_0"]))
# )

Classifier_type,allele_0,Allele_set,Batch,AUROC_thresh,AUROC_mean,trainsize_0_mean,testsize_0_mean,trainsize_1_mean,testsize_1_mean,Training_imbalance_mean,morph_change
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,i32
"""morphology""","""HBD_Gly26Asp""","""A1P2""","""B7""",0.997315,0.994581,1640.666667,648.333333,621.666667,202.333333,2.644345,0
"""morphology""","""PYGL_Asn339Ser""","""A1P3""","""B8""",0.99807,0.459475,230.666667,115.333333,185.333333,92.666667,1.240988,0
"""morphology""","""AGXT_Ala186Val""","""A1P1""","""B7""",0.997315,0.852631,1099.5,366.5,1590.75,530.25,1.448952,0
"""morphology""","""PKP2_Tyr582Asn""","""A1P3""","""B8""",0.99807,0.466434,111.333333,55.666667,78.666667,39.333333,1.635455,0
"""morphology""","""PKP2_Thr482Met""","""A1P3""","""B7""",0.997315,0.689049,645.666667,262.333333,395.333333,71.666667,1.635137,0


Classifier_type,allele_0,Allele_set,Batch,AUROC_thresh,AUROC_mean,trainsize_0_mean,testsize_0_mean,trainsize_1_mean,testsize_1_mean,Training_imbalance_mean,mislocalized
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,i32
"""localization""","""AGXT_Ala210Pro""","""A1P1""","""B7""",0.71418,0.932789,1238.25,412.75,1590.75,530.25,1.283454,1
"""localization""","""AGXT_Gly161Arg""","""A1P1""","""B8""",0.861392,0.80332,265.5,88.5,440.25,146.75,1.734075,0
"""localization""","""COMP_Arg718Pro""","""A1P1""","""B8""",0.861392,0.968708,369.75,123.25,201.0,67.0,1.847131,1
"""localization""","""KRT86_Ala118Glu""","""A1P2""","""B8""",0.861392,0.692461,208.5,69.5,329.25,109.75,1.598304,0
"""localization""","""TH_Ile209Met""","""A1P4""","""B8""",0.861392,0.629974,259.5,86.5,193.5,64.5,1.336459,0


## Plot Variant Painting Images

In [4]:
# Define mapping between simple names and folder names
batch_dict = {
    "B7A1R1": "2024_01_23_Batch_7",
    "B7A2R1": "2024_01_23_Batch_7",
    "B8A1R2": "2024_02_06_Batch_8",
    "B8A2R2": "2024_02_06_Batch_8"
}

letter_dict = {
    "A": "01",
    "B": "02",
    "C": "03",
    "D": "04",
    "E": "05",
    "F": "06",
    "G": "07",
    "H": "08",
    "I": "09",
    "J": "10",
    "K": "11",
    "L": "12",
    "M": "13",
    "N": "14",
    "O": "15",
    "P": "16",
}

plate_dict = {    
    "B7A1R1_P1": {"T1": "2024_01_17_B7A1R1_P1T1__2024_01_17T08_35_58_Measurement_1",
                  "T2": "2024_01_17_B7A1R1_P1T2__2024_01_17T10_13_45_Measurement_1",
                  "T3": "2024_01_17_B7A1R1_P1T3__2024_01_17T11_58_08_Measurement_1",
                  "T4": "2024_01_17_B7A1R1_P1T4__2024_01_17T13_45_14_Measurement_1"},
    
    "B7A1R1_P2": {"T1": "2024_01_17_B7A1R1_P2T1__2024_01_17T15_33_09_Measurement_1",
                  "T2": "2024_01_17_B7A1R1_P2T2__2024_01_18T08_25_01_Measurement_1",
                  "T3": "2024_01_17_B7A1R1_P2T3__2024_01_18T10_47_36_Measurement_1",
                  "T4": "2024_01_17_B7A1R1_P2T4__2024_01_18T12_48_20_Measurement_1"},
        
    "B7A1R1_P3": {"T1": "2024_01_18_B7A1R1_P3T1__2024_01_18T14_27_08_Measurement_1",
                  "T2": "2024_01_19_B7A1R1_P3T2__2024_01_19T08_23_30_Measurement_1",
                  "T3": "2024_01_19_B7A1R1_P3T3__2024_01_19T10_01_45_Measurement_1",
                  "T4": "2024_01_19_B7A1R1_P3T4__2024_01_19T12_00_10_Measurement_1"},
            
    "B7A1R1_P4": {"T1": "2024_01_19_B7A1R1_P4T1__2024_01_19T13_50_55_Measurement_1",
                  "T2": "2024_01_23_B7A1R1_P4T2__2024_01_23T10_13_00_Measurement_1",
                  "T3": "2024_01_22_B7A1R1_P4T3__2024_01_22T08_37_41_Measurement_1",
                  "T4": "2024_01_22_B7A1R1_P4T4__2024_01_22T10_27_16_Measurement_1"},
    
    "B7A2R1_P1": {"T1": "2024_01_22_B7A2R1_P1T1__2024_01_22T12_13_13_Measurement_1",
                  "T2": "2024_01_22_B7A2R1_P1T2__2024_01_22T13_52_24_Measurement_1",
                  "T3": "2024_01_22_B7A2R1_P1T3__2024_01_22T15_29_31_Measurement_1",
                  "T4": "2024_01_23_B7A2R1_P1T4__2024_01_23T08_28_07_Measurement_1"},
    
    "B8A1R2_P1": {"T1": "2024_01_31_B8A1R2_P1T1__2024_01_31T10_11_57_Measurement_1",
                  "T2": "2024_01_31_B8A1R2_P1T2__2024_01_31T08_35_51_Measurement_2",
                  "T3": "2024_01_31_B8A1R2_P1T3__2024_01_31T12_09_14_Measurement_1",
                  "T4": "2024_01_31_B8A1R2_P1T4__2024_01_31T14_02_18_Measurement_2"},
    
    "B8A1R2_P2": {"T1": "2024_01_31_B8A1R2_P2T1__2024_01_31T15_41_23_Measurement_1",
                  "T2": "2024_02_01_B8A1R2_P2T2__2024_02_01T10_23_20_Measurement_2",
                  "T3": "2024_02_01_B8A1R2_P2T3__2024_02_01T12_16_30_Measurement_4",
                  "T4": "2024_02_01_B8A1R2_P2T4__2024_02_01T14_05_52_Measurement_1"},
        
    "B8A1R2_P3": {"T1": "2024_02_02_B8A1R2_P3T1__2024_02_02T08_32_30_Measurement_2",
                  "T2": "2024_02_02_B8A1R2_P3T2__2024_02_02T10_08_05_Measurement_1",
                  "T3": "2024_02_02_B8A1R2_P3T3__2024_02_02T11_58_46_Measurement_2",
                  "T4": "2024_02_02_B8A1R2_P3T4__2024_02_02T13_51_50_Measurement_1"},
            
    "B8A1R2_P4": {"T1": "2024_02_02_B8A1R2_P4T1__2024_02_02T15_32_28_Measurement_1",
                  "T2": "2024_02_05_B8A1R2_P4T2__2024_02_05T08_22_47_Measurement_2",
                  "T3": "2024_02_05_B8A1R2_P4T3__2024_02_05T10_00_30_Measurement_1",
                  "T4": "2024_02_05_B8A1R2_P4T4__2024_02_05T11_38_50_Measurement_1"},
    
    "B8A2R2_P1": {"T1": "2024_02_05_B8A2R2_P1T1__2024_02_05T13_23_28_Measurement_1",
                  "T2": "2024_02_05_B8A2R2_P1T2__2024_02_05T15_18_32_Measurement_1",
                  "T3": "2024_02_06_B8A2R2_P1T3__2024_02_06T08_23_49_Measurement_1",
                  "T4": "2024_02_06_B8A2R2_P1T4__2024_02_06T10_18_16_Measurement_1"}
}

channel_dict = {
    "DAPI": "1",
    "GFP": "2",
    "AGP": "3",
    "Mito": "4",
    "Brightfield1": "5",
    "Brightfield2": "6",
    "Brightfield": "7",
}

In [5]:
tiff_imgs_dir = f"{CP_GALLERY_DIR}/tiff_images/cpg0020-varchamp/broad/images"
out_imgs_dir = "../snakemake_pipeline/outputs/visualize_imgs"

In [6]:
def channel_to_cmap(channel):
    if channel == "GFP":
        cmap = mpl.colors.LinearSegmentedColormap.from_list("green_cmap", ["#000","#65fe08"])
    elif channel == "DAPI":
        cmap = mpl.colors.LinearSegmentedColormap.from_list("green_cmap", ["#000","#0000FF"])
    elif channel == "Mito":
        cmap = mpl.colors.LinearSegmentedColormap.from_list("green_cmap", ["#000","#FF0000"]) 
    elif channel == "AGP":
        cmap = mpl.colors.LinearSegmentedColormap.from_list("green_cmap", ["#000","#FFFF00"]) 
    else:
        cmap = "gray"
    return cmap


# Convert letter rows to numbers
def well_to_coordinates(well):
    row_letter, col_number = re.match(r"([A-P])(\d{2})", well).groups()
    row_index = ord(row_letter) - ord('A') + 1  # Convert 'A'->1, 'B'->2, ..., 'P'->16
    col_index = int(col_number)  # Convert string column to integer
    return well, row_index, col_index


# Compute distances from edges and find the most centered well
def compute_distance(row, col):
    return min(row - 1, 16 - row, col - 1, 24 - col)  # Distance from nearest edge
    

def plot_allele(pm, variant, sel_channel, auroc_df, site="05", max_intensity=0.99, display=False, imgs_dir=tiff_imgs_dir, output_dir=out_imgs_dir):
    cmap = channel_to_cmap(sel_channel)
    channel = channel_dict[sel_channel]
    auroc = auroc_df.filter(pl.col("allele_0")==variant)["AUROC_mean"].mean()
    if os.path.exists(os.path.join(output_dir, f"{variant}_{sel_channel}_{auroc:.3f}.png")):
        print(f"Image for {variant} already exists.")
        return None
        
    ## get the number of wells/images per allele
    wt = variant.split("_")[0]
    wt_wells = pm.filter(pl.col("gene_allele") == wt).select("imaging_well").to_pandas().values.flatten()
    var_wells = pm.filter(pl.col("gene_allele") == variant).select("imaging_well").to_pandas().values.flatten()
    plate_map = pm.filter(pl.col("gene_allele") == variant).select("plate_map_name").to_pandas().values.flatten()

    if len(wt_wells) > 1:
        # Get coordinates of wells
        well_coords = [well_to_coordinates(w) for w in set([ref_well_pl for ref_well_pl in wt_wells])]
        # Sort wells by max distance from edges (descending)
        wt_wells = [max(well_coords, key=lambda x: compute_distance(x[1], x[2]))[0]]

    pm_var = pm.filter((pl.col("imaging_well").is_in(np.concatenate([wt_wells, var_wells])))&(pl.col("plate_map_name").is_in(plate_map))).sort("control_type")
    # print(pm_var)
    plt.clf()

    fig, axes = plt.subplots((len(wt_wells)+len(var_wells))*2, 4, figsize=(15, 16), sharex=True, sharey=True)
    for wt_var, pm_row in enumerate(pm_var.iter_rows(named=True)):
        # print(pm_row)
        if pm_row["control_type"] == "allele":
            well = var_wells[0]
            allele = variant
        else:
            well = wt_wells[0]
            allele = wt

        for i in range(8):
            if i < 4:
                sel_plate = pm_row["imaging_plate_R1"]
            else:
                sel_plate = pm_row["imaging_plate_R2"]
                
            batch = batch_dict[sel_plate.split("_")[0]]
            batch_img_dir = f'{imgs_dir}/{batch}/images'
            
            letter = well[0]
            row = letter_dict[letter]
            col = well[1:3]
            
            plate_img_dir = plate_dict[sel_plate][f"T{i%4+1}"]
            img_file = f"r{row}c{col}f{site}p01-ch{channel}sk1fk1fl1.tiff"
            img = imread(f"{batch_img_dir}/{plate_img_dir}/Images/{img_file}", as_gray=True)
            
            # Get key intensity percentiles
            p99 = np.percentile(img, 99)
            p100 = np.max(img)

            # If there's a big jump between 99th and max, lower the threshold
            if p100 > 1.5 * p99:
                # print("More contrast")
                max_intensity = 0.95  # More contrast
            else:
                max_intensity = max_intensity  # Allow more of the bright tail

            plot_idx = i+wt_var*4*2
            axes.flatten()[plot_idx].imshow(img, vmin=0, vmax=np.percentile(img, max_intensity*100), cmap=cmap)
            plot_label = f"{sel_channel}:{sel_plate},T{i%4+1}\nWell:{well},Site:{site}\n{allele}"
            axes.flatten()[plot_idx].text(0.03, 0.97, plot_label, color='white', fontsize=10,
                    verticalalignment='top', horizontalalignment='left', transform=axes.flatten()[plot_idx].transAxes,
                    bbox=dict(facecolor='black', alpha=0.3, linewidth=2))
            int_95 = str(int(round(np.percentile(img, 95))))
            axes.flatten()[plot_idx].text(0.95, 0.05, f"95th Percentile\nIntensity:{int_95}", color='white', fontsize=12,
                           verticalalignment='bottom', horizontalalignment='right', transform=axes.flatten()[plot_idx].transAxes)
            axes.flatten()[plot_idx].axis("off")
        
    plt.tight_layout()
    plt.subplots_adjust(wspace=.01, hspace=-0.2, top=.99)
    if display:
        plt.show()
    if output_dir:
        plt.savefig(os.path.join(output_dir, f"{variant}_{sel_channel}_{auroc:.3f}.png"), dpi=400, bbox_inches='tight')
        plt.close(fig)

In [7]:
# plot_allele(meta_data, "SMAD1_Val3Ala", "GFP", auroc_df=local_wtvar, site="05", max_intensity=0.999, display=True, imgs_dir=tiff_imgs_dir, output_dir=out_imgs_dir)

In [8]:
# variant_list = ['ACSF3_Met266Val',
#  'ACSF3_Gly119Asp',
#  'ACSF3_Arg558Trp',
#  'ACSF3_Arg471Trp',
#  'ACSF3_Pro243Leu',
#  'ACSF3_Asp457Asn',
#  'ACSF3_Thr358Ile']

# variant_list = ['PLA2G6_Thr585Ile',
#             'PLA2G6_Leu481Gln',
#             'PLA2G6_Arg600Gln',
#             'PLA2G6_Val310Glu',
#             'PLA2G6_Glu567Lys',
#             'PLA2G6_Arg538Cys']

# variant_list = ['IL2RG_Tyr89Cys', 'KCTD7_Asn273Ile', 'AGXT_Ile279Met',
#        'ASNS_Ala6Glu', 'ACSF3_Ser431Tyr', 'HBD_Gly26Asp',
#        'SFTPC_Leu188Gln', 'SH3BP2_Gly420Glu', 'DIABLO_Ser126Leu',
#        'PLP1_Ser135Phe', 'DES_Ala337Pro', 'SFTPC_Leu194Pro',
#        'EIF2B4_Ala228Val', 'TH_Pro121Leu', 'EFHC1_Asp253Tyr',
#        'FBP1_Phe194Ser', 'POMT1_Trp560Cys', 'AIPL1_Met79Thr',
#        'EFHC1_Cys259Tyr', 'AGXT_Ile202Asn', 'SMAD1_Val3Ala',
#        'RP2_Cys86Tyr', 'EFHC1_Met448Thr', 'SFTPC_Ile73Thr',
#        'SMN2_Gly287Arg', 'EFHC1_Asp210Asn', 'PKP2_Ala526Thr',
#        'RP2_Leu253Arg', 'PLP1_Ile152Thr', 'PMP22_Asp37Val',
#        'PLP1_Gly186Cys', 'TAT_Arg433Trp', 'TTPA_Gly246Arg',
#        'DCX_Ser129Leu', 'COMP_Asn523Lys', 'FXN_Trp173Gly']

# for variant in tqdm(variant_list):
#     plot_allele(meta_data, variant, "GFP", auroc_df=local_wtvar, site="05", max_intensity=0.999, display=False, imgs_dir=tiff_imgs_dir, output_dir=out_imgs_dir)

## Plot Cells

In [9]:
out_cells_dir = "../snakemake_pipeline/outputs/visualize_cells"

In [24]:
# Filter thresholds
min_area_ratio = 0.15
max_area_ratio = 0.3
min_center = 50
max_center = 1030
num_mad = 5
min_cells = 250

batch_profiles = {}
for batch_id in batches:
    imagecsv_dir = f"{CP_GALLERY_DIR}/image_csv/cpg0020-varchamp/broad/workspace/assaydev/{batch_id}"
    prof_path = f"../snakemake_pipeline/outputs/batch_profiles/{batch_id}/profiles.parquet"
    # Get metadata
    profiles = pl.scan_parquet(prof_path).select(
        ["Metadata_well_position", "Metadata_plate_map_name", "Metadata_ImageNumber", "Metadata_ObjectNumber",
            "Metadata_symbol", "Metadata_gene_allele", "Metadata_control_type", "Metadata_Plate",
            "Nuclei_AreaShape_Area", "Cells_AreaShape_Area", "Nuclei_AreaShape_Center_X", "Nuclei_AreaShape_Center_Y",
            "Cells_AreaShape_BoundingBoxMaximum_X", "Cells_AreaShape_BoundingBoxMaximum_Y", "Cells_AreaShape_BoundingBoxMinimum_X",
            "Cells_AreaShape_BoundingBoxMinimum_Y",	"Cells_AreaShape_Center_X",	"Cells_AreaShape_Center_Y",
            "Cells_Intensity_MeanIntensity_GFP", "Cells_Intensity_MedianIntensity_GFP", "Cells_Intensity_IntegratedIntensity_GFP"],
    ).collect()

    # Filter based on cell to nucleus area
    profiles = profiles.with_columns(
                    (pl.col("Nuclei_AreaShape_Area")/pl.col("Cells_AreaShape_Area")).alias("Nucleus_Cell_Area"),
                    pl.concat_str([
                        "Metadata_Plate", "Metadata_well_position", "Metadata_ImageNumber", "Metadata_ObjectNumber",
                        ], separator="_").alias("Metadata_CellID"),
            ).filter((pl.col("Nucleus_Cell_Area") > min_area_ratio) & (pl.col("Nucleus_Cell_Area") < max_area_ratio))

    # Filter cells too close to image edge
    profiles = profiles.filter(
        ((pl.col("Nuclei_AreaShape_Center_X") > min_center) & (pl.col("Nuclei_AreaShape_Center_X") < max_center) &
        (pl.col("Nuclei_AreaShape_Center_Y") > min_center) & (pl.col("Nuclei_AreaShape_Center_Y") < max_center)),
    )

    # Calculate mean, median and mad of gfp intensity for each allele
    ## mean
    means = profiles.group_by(["Metadata_Plate", "Metadata_well_position"]).agg(
        pl.col("Cells_Intensity_MeanIntensity_GFP").mean().alias("WellIntensityMean"),
    )
    profiles = profiles.join(means, on=["Metadata_Plate", "Metadata_well_position"])
    ## median
    medians = profiles.group_by(["Metadata_Plate", "Metadata_well_position"]).agg(
        pl.col("Cells_Intensity_MedianIntensity_GFP").median().alias("WellIntensityMedian"),
    )
    profiles = profiles.join(medians, on=["Metadata_Plate", "Metadata_well_position"])
    ## mad
    profiles = profiles.with_columns(
        (pl.col("Cells_Intensity_MedianIntensity_GFP") - pl.col("WellIntensityMedian")).abs().alias("Abs_dev"),
    )
    mad = profiles.group_by(["Metadata_Plate", "Metadata_well_position"]).agg(
        pl.col("Abs_dev").median().alias("Intensity_MAD"),
    )
    profiles = profiles.join(mad, on=["Metadata_Plate", "Metadata_well_position"])

    # Threshold is 5X
    ## Used to be median well intensity + 5*mad implemented by Jess
    ## Switching to mean well intensity + 5*mad implemented by Runxi
    # profiles = profiles.with_columns(
    #     (pl.col("WellIntensityMean") + num_mad*pl.col("Intensity_MAD")).alias("Intensity_upper_threshold"), ## pl.col("WellIntensityMedian")
    #     (pl.col("WellIntensityMean") - num_mad*pl.col("Intensity_MAD")).alias("Intensity_lower_threshold"), ## pl.col("WellIntensityMedian")
    # )
    # ## Filter by intensity MAD
    # profiles = profiles.filter(
    #     pl.col("Cells_Intensity_MeanIntensity_GFP") <= pl.col("Intensity_upper_threshold"),
    # ).filter(
    #     pl.col("Cells_Intensity_MeanIntensity_GFP") >= pl.col("Intensity_lower_threshold"),
    # )

    # Filter out alleles with fewer than 250 cells
    # keep_alleles = profiles.group_by("Metadata_gene_allele").count().filter(
    #     pl.col("count") >= min_cells,
    #     ).select("Metadata_gene_allele").to_series().to_list()
    # profiles = profiles.filter(pl.col("Metadata_gene_allele").is_in(keep_alleles))

    # add full crop coordinates
    profiles = profiles.with_columns(
        (pl.col("Nuclei_AreaShape_Center_X") - 50).alias("x_low").round().cast(pl.Int16),
        (pl.col("Nuclei_AreaShape_Center_X") + 50).alias("x_high").round().cast(pl.Int16),
        (pl.col("Nuclei_AreaShape_Center_Y") - 50).alias("y_low").round().cast(pl.Int16),
        (pl.col("Nuclei_AreaShape_Center_Y") + 50).alias("y_high").round().cast(pl.Int16),
    )

    # Read in all Image.csv to get ImageNumber:SiteNumber mapping and paths
    image_dat = []
    icf = os.listdir(imagecsv_dir)
    for fp in tqdm(icf):
        plate, well = fp.split("-")
        image_dat.append(pl.read_csv(f"{imagecsv_dir}/{fp}/Image.csv").select(
            [
                "ImageNumber",
                "Metadata_Site",
                "PathName_OrigDNA",
                "FileName_OrigDNA",
                "FileName_OrigGFP",
                ],
            ).with_columns(
            pl.lit(plate).alias("Metadata_Plate"),
            pl.lit(well).alias("Metadata_well_position"),
            ))
    image_dat = pl.concat(image_dat).rename({"ImageNumber": "Metadata_ImageNumber"})

    # Create useful filepaths
    image_dat = image_dat.with_columns(
        pl.col("PathName_OrigDNA").str.replace(".*cpg0020-varchamp/", "").alias("Path_root"),
    )

    image_dat = image_dat.drop([
        "PathName_OrigDNA",
        "FileName_OrigDNA",
        "FileName_OrigGFP",
        "Path_root",
    ])

    # Append to profiles
    profiles = profiles.join(image_dat, on = ["Metadata_Plate", "Metadata_well_position", "Metadata_ImageNumber"])

    # Sort by allele, then image number
    profiles = profiles.with_columns(
        pl.concat_str(["Metadata_Plate", "Metadata_well_position", "Metadata_Site"], separator="_").alias(
            "Metadata_SiteID"),
        pl.col("Metadata_gene_allele").str.replace("_", "-").alias("Protein_label"),
    )
    profiles = profiles.sort(["Protein_label", "Metadata_SiteID"])
    alleles = profiles.select("Protein_label").to_series().unique().to_list()

    batch_profiles[batch_id] = profiles

100%|██████████| 7665/7665 [00:06<00:00, 1229.72it/s]


In [15]:
def crop_allele(allele: str, profile_df: pl.DataFrame, meta_plate: str, rep: str, site: str="5") -> None:
    """Crop images and save metadata as numpy arrays for one allele.

    Parameters
    ----------
    allele : String
        Name of allele to process
    profile_df : String
        Dataframe with pathname and cell coordinates
    img_dir : String
        Directory where all images are stored
    out_dir : String
        Directory where numpy arrays should be saved
    """
    allele_df = profile_df.filter(
        (pl.col("Metadata_gene_allele")==allele) &
        (pl.col("Metadata_Site")==int(site)) &
        (pl.col("Metadata_plate_map_name")==meta_plate) &
        (pl.col("Metadata_Plate").str.contains(rep))
    )
    return allele_df

allele_df = crop_allele("SMAD1", batch_profiles["2024_01_23_Batch_7"], "B7A1R1_P4", "T1", site="5")
allele_df

Metadata_well_position,Metadata_plate_map_name,Metadata_ImageNumber,Metadata_ObjectNumber,Metadata_symbol,Metadata_gene_allele,Metadata_control_type,Metadata_Plate,Nuclei_AreaShape_Area,Cells_AreaShape_Area,Nuclei_AreaShape_Center_X,Nuclei_AreaShape_Center_Y,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_Intensity_MeanIntensity_GFP,Cells_Intensity_MedianIntensity_GFP,Cells_Intensity_IntegratedIntensity_GFP,Nucleus_Cell_Area,Metadata_CellID,WellIntensityMean,WellIntensityMedian,Abs_dev,Intensity_MAD,Intensity_upper_threshold,Intensity_lower_threshold,x_low,x_high,y_low,y_high,Metadata_Site,Metadata_SiteID,Protein_label
str,str,i64,i64,str,str,str,str,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,i16,i16,i16,i16,i64,str,str
"""A05""","""B7A1R1_P4""",41,11,"""SMAD1""","""SMAD1""","""disease_wt""","""2024_01_19_B7A1R1_P4T1""",1096,5814,81.438869,415.609489,168,458,58,359,96.54128,413.690918,0.006644,0.00633,38.629356,0.18851,"""2024_01_19_B7A1R1_P4T1_A05_41_…",0.004921,0.003691,0.002638,0.001137,0.010605,-0.000763,31,131,366,466,5,"""2024_01_19_B7A1R1_P4T1_A05_5""","""SMAD1"""
"""A05""","""B7A1R1_P4""",41,24,"""SMAD1""","""SMAD1""","""disease_wt""","""2024_01_19_B7A1R1_P4T1""",1809,11405,88.215589,704.983416,153,743,0,633,71.303639,693.142744,0.010516,0.010121,119.936142,0.158615,"""2024_01_19_B7A1R1_P4T1_A05_41_…",0.004921,0.003691,0.006429,0.001137,0.010605,-0.000763,38,138,655,755,5,"""2024_01_19_B7A1R1_P4T1_A05_5""","""SMAD1"""
"""A05""","""B7A1R1_P4""",41,18,"""SMAD1""","""SMAD1""","""disease_wt""","""2024_01_19_B7A1R1_P4T1""",1153,6510,881.117953,631.679965,943,668,842,533,891.28894,614.277266,0.003122,0.003021,20.327037,0.177112,"""2024_01_19_B7A1R1_P4T1_A05_41_…",0.004921,0.003691,0.00067,0.001137,0.010605,-0.000763,831,931,582,682,5,"""2024_01_19_B7A1R1_P4T1_A05_5""","""SMAD1"""
"""A05""","""B7A1R1_P4""",41,28,"""SMAD1""","""SMAD1""","""disease_wt""","""2024_01_19_B7A1R1_P4T1""",878,4949,604.738041,805.56492,635,858,556,759,593.280865,801.839968,0.003648,0.003634,18.054927,0.17741,"""2024_01_19_B7A1R1_P4T1_A05_41_…",0.004921,0.003691,0.000057,0.001137,0.010605,-0.000763,555,655,756,856,5,"""2024_01_19_B7A1R1_P4T1_A05_5""","""SMAD1"""
"""A05""","""B7A1R1_P4""",41,29,"""SMAD1""","""SMAD1""","""disease_wt""","""2024_01_19_B7A1R1_P4T1""",877,3563,937.348917,805.080958,965,837,878,776,921.106652,801.920573,0.004,0.003816,14.252918,0.246141,"""2024_01_19_B7A1R1_P4T1_A05_41_…",0.004921,0.003691,0.000124,0.001137,0.010605,-0.000763,887,987,755,855,5,"""2024_01_19_B7A1R1_P4T1_A05_5""","""SMAD1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""A05""","""B7A1R1_P4""",41,23,"""SMAD1""","""SMAD1""","""disease_wt""","""2024_01_19_B7A1R1_P4T1""",1214,7037,874.387974,701.83855,933,776,829,666,875.765809,722.628393,0.002756,0.002659,19.395149,0.172517,"""2024_01_19_B7A1R1_P4T1_A05_41_…",0.004921,0.003691,0.001033,0.001137,0.010605,-0.000763,824,924,652,752,5,"""2024_01_19_B7A1R1_P4T1_A05_5""","""SMAD1"""
"""A05""","""B7A1R1_P4""",41,32,"""SMAD1""","""SMAD1""","""disease_wt""","""2024_01_19_B7A1R1_P4T1""",1511,9829,124.005295,840.509596,179,920,33,786,110.007529,844.244277,0.003028,0.002951,29.76286,0.153729,"""2024_01_19_B7A1R1_P4T1_A05_41_…",0.004921,0.003691,0.00074,0.001137,0.010605,-0.000763,74,174,791,891,5,"""2024_01_19_B7A1R1_P4T1_A05_5""","""SMAD1"""
"""A05""","""B7A1R1_P4""",41,34,"""SMAD1""","""SMAD1""","""disease_wt""","""2024_01_19_B7A1R1_P4T1""",1899,10988,239.512902,877.313323,327,1033,103,800,206.159629,916.411631,0.008434,0.008457,92.670951,0.172825,"""2024_01_19_B7A1R1_P4T1_A05_41_…",0.004921,0.003691,0.004765,0.001137,0.010605,-0.000763,190,290,827,927,5,"""2024_01_19_B7A1R1_P4T1_A05_5""","""SMAD1"""


In [20]:
# Compute distances from edges and find the most centered well
def compute_distance_cell(row, col, edge=1080):
    return min(row - 1, edge - row, col - 1, edge - col)  # Distance from nearest edge
    

def plot_allele_cell(pm, variant, sel_channel, auroc_df, site="05", max_intensity=0.99, display=False, imgs_dir=tiff_imgs_dir, output_dir=out_imgs_dir):
    cmap = channel_to_cmap(sel_channel)
    channel = channel_dict[sel_channel]
    auroc = auroc_df.filter(pl.col("allele_0")==variant)["AUROC_mean"].mean()

    if os.path.exists(os.path.join(output_dir, f"{variant}_{sel_channel}_{auroc:.3f}_cells.png")):
        print(f"Image for {variant} already exists.")
        # return None

    ## get the number of wells/images per allele
    wt = variant.split("_")[0]
    wt_wells = pm.filter(pl.col("gene_allele") == wt).select("imaging_well").to_pandas().values.flatten()
    var_wells = pm.filter(pl.col("gene_allele") == variant).select("imaging_well").to_pandas().values.flatten()
    plate_map = pm.filter(pl.col("gene_allele") == variant).select("plate_map_name").to_pandas().values.flatten()

    if len(wt_wells) > 1:
        # Get coordinates of wells
        well_coords = [well_to_coordinates(w) for w in set([ref_well_pl for ref_well_pl in wt_wells])]
        # Sort wells by max distance from edges (descending)
        wt_wells = [max(well_coords, key=lambda x: compute_distance(x[1], x[2]))[0]]

    pm_var = pm.filter((pl.col("imaging_well").is_in(np.concatenate([wt_wells, var_wells])))&(pl.col("plate_map_name").is_in(plate_map))).sort("control_type")
    # print(pm_var)
    plt.clf()

    fig, axes = plt.subplots((len(wt_wells)+len(var_wells))*2, 4, figsize=(15, 16), sharex=True, sharey=True)
    # fig2, axes2 = plt.subplots((len(wt_wells)+len(var_wells))*2, 4, figsize=(15, 16), sharex=True, sharey=True)
    for wt_var, pm_row in enumerate(pm_var.iter_rows(named=True)):
        # print(pm_row)
        if pm_row["control_type"] == "allele":
            well = var_wells[0]
            allele = variant
        else:
            well = wt_wells[0]
            allele = wt

        for i in range(8):
            plot_idx = i+wt_var*4*2
            if i < 4:
                sel_plate = pm_row["imaging_plate_R1"]
            else:
                sel_plate = pm_row["imaging_plate_R2"]
                
            batch = batch_dict[sel_plate.split("_")[0]]
            batch_img_dir = f'{imgs_dir}/{batch}/images'
            letter = well[0]
            row = letter_dict[letter]
            col = well[1:3]
            
            # print(i, allele, well)
            plate_img_dir = plate_dict[sel_plate][f"T{i%4+1}"]
            img_file = f"r{row}c{col}f{site}p01-ch{channel}sk1fk1fl1.tiff"
            img = imread(f"{batch_img_dir}/{plate_img_dir}/Images/{img_file}", as_gray=True)

            ## Full images
            # axes2.flatten()[plot_idx].imshow(img, vmin=0, vmax=np.percentile(img, max_intensity*100), cmap=cmap)
            # plot_label = f"{sel_channel}:{sel_plate},T{i%4+1}\nWell:{well},Site:{site}\n{allele}"
            # axes2.flatten()[plot_idx].text(0.03, 0.97, plot_label, color='white', fontsize=10,
            #         verticalalignment='top', horizontalalignment='left', transform=axes2.flatten()[plot_idx].transAxes,
            #         bbox=dict(facecolor='black', alpha=0.3, linewidth=2))

            ## Draw cells
            cell_allele_coord_df = crop_allele(allele, batch_profiles[batch], sel_plate, f"T{i%4+1}", site=site[-1]) # "2024_01_19_B7A1R1_P4T1"
            cell_allele_coord_df = cell_allele_coord_df.with_columns(
                pl.struct("Cells_AreaShape_Center_X", "Cells_AreaShape_Center_Y") # 'Nuclei_AreaShape_Center_X', 'Nuclei_AreaShape_Center_Y'
                .map_elements(lambda x: compute_distance_cell(x['Cells_AreaShape_Center_X'], x['Cells_AreaShape_Center_Y']), return_dtype=pl.Float32).cast(pl.Int16)
                .alias('dist2edge')
            ).sort("dist2edge", descending=True)
            
            if cell_allele_coord_df.is_empty():
                axes.flatten()[plot_idx].set_visible(False)
                print(i, allele, well)
                continue
            else:
                x, y = int(cell_allele_coord_df["Cells_AreaShape_Center_X"].to_numpy()[0]), int(cell_allele_coord_df["Cells_AreaShape_Center_Y"].to_numpy()[0])
                # print(x, y)
                img_sub = img[
                    y-128:y+128, x-128:x+128
                ]
                # print(img_sub.shape)
                if img_sub.shape[0] == 0 or img_sub.shape[1] == 0:
                    axes.flatten()[plot_idx].set_visible(False)
                    continue

                # print(img_sub.shape)
                # Get key intensity percentiles
                p99 = np.percentile(img_sub, 99)
                p100 = np.max(img_sub)

                # If there's a big jump between 99th and max, lower the threshold
                if p100 > 1.5 * p99:
                    # print("More contrast")
                    max_intensity = 0.95  # More contrast
                else:
                    max_intensity = max_intensity  # Allow more of the bright tail

                axes.flatten()[plot_idx].imshow(img_sub, vmin=0, vmax=np.percentile(img_sub, max_intensity*100), cmap=cmap)  ## np.percentile(img_sub, max_intensity*100)
                plot_label = f"{sel_channel}:{sel_plate},T{i%4+1}\nWell:{well},Site:{site}\n{allele}"
                axes.flatten()[plot_idx].text(0.03, 0.97, plot_label, color='white', fontsize=10,
                        verticalalignment='top', horizontalalignment='left', transform=axes.flatten()[plot_idx].transAxes,
                        bbox=dict(facecolor='black', alpha=0.3, linewidth=2))
                int_95 = str(int(round(np.percentile(img_sub, 95))))
                axes.flatten()[plot_idx].text(0.95, 0.05, f"95th Percentile\nIntensity:{int_95}", color='white', fontsize=12,
                            verticalalignment='bottom', horizontalalignment='right', transform=axes.flatten()[plot_idx].transAxes,
                            bbox=dict(facecolor='black', alpha=0.3, linewidth=2))
                axes.flatten()[plot_idx].axis("off")
                
                # x_min, y_min, x_max, y_max = int(cell_allele_coord_df["Cells_AreaShape_BoundingBoxMinimum_X"].to_numpy()[0]), \
                #                              int(cell_allele_coord_df["Cells_AreaShape_BoundingBoxMinimum_Y"].to_numpy()[0]), \
                #                              int(cell_allele_coord_df["Cells_AreaShape_BoundingBoxMaximum_X"].to_numpy()[0]), \
                #                              int(cell_allele_coord_df["Cells_AreaShape_BoundingBoxMaximum_Y"].to_numpy()[0])
                # print(x_min, y_min, x_max, y_max)
            #     for cell_idx, cell_row in enumerate(cell_allele_coord_df.head().iter_rows(named=True)):
            #         x_nuc, y_nuc = int(cell_row["Nuclei_AreaShape_Center_X"]), int(cell_row["Nuclei_AreaShape_Center_Y"])
            #         # Create a Rectangle patch
            #         rect = patches.Rectangle((x_nuc-5, y_nuc-5), 10, 10, linewidth=2, edgecolor='red', facecolor='none')
            #         axes2.flatten()[plot_idx].add_patch(rect)
            #         rect = patches.Rectangle((x-128, y-128), 256, 256, linewidth=2, edgecolor='red', facecolor='none')
            #         axes2.flatten()[plot_idx].add_patch(rect)
            #         # x_min, y_min, x_max, y_max = int(cell_row["Cells_AreaShape_BoundingBoxMinimum_X"]), int(cell_row["Cells_AreaShape_BoundingBoxMinimum_Y"]), \
            #         #                              int(cell_row["Cells_AreaShape_BoundingBoxMaximum_X"]), int(cell_row["Cells_AreaShape_BoundingBoxMaximum_Y"])
            #         # print(x_min, y_min, x_max, y_max)
            #         # rect = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=2, edgecolor='red', facecolor='none')
            #         # Add the patch to the Axes
            #         axes2.flatten()[plot_idx].add_patch(rect)
            #         break

            # int_95 = str(int(round(np.percentile(img, 95))))
            # axes2.flatten()[plot_idx].axhline(y=img.shape[1]//2, color='red', linestyle='--', linewidth=1)
            # axes2.flatten()[plot_idx].axvline(x=img.shape[0]//2, color='red', linestyle='--', linewidth=1)
            # axes2.flatten()[plot_idx].text(0.95, 0.05, f"95th Percentile\nIntensity:{int_95}", color='white', fontsize=12,
            #                verticalalignment='bottom', horizontalalignment='right', transform=axes2.flatten()[plot_idx].transAxes)
            # axes2.flatten()[plot_idx].axis("off")
            
    
    fig.tight_layout()
    fig.subplots_adjust(wspace=.01, hspace=-0.2, top=.99)

    # fig2.tight_layout()
    # fig2.subplots_adjust(wspace=.01, hspace=-0.2, top=.99)
    
    if display:
        plt.show()

    if output_dir:
        fig.savefig(os.path.join(output_dir, f"{variant}_{sel_channel}_{auroc:.3f}_cells.png"), dpi=400, bbox_inches='tight')
        plt.close(fig)

plot_allele_cell(meta_data, "SMAD1_Val3Ala", "GFP", auroc_df=local_wtvar, site="05", max_intensity=0.95, display=False, imgs_dir=tiff_imgs_dir, output_dir=out_cells_dir)

Image for SMAD1_Val3Ala already exists.
7 SMAD1 A05


In [14]:
# plot_allele_cell(meta_data, "PLA2G6_Val310Glu", "GFP", auroc_df=local_wtvar, site="05", max_intensity=0.99, display=True, imgs_dir=tiff_imgs_dir, output_dir=out_cells_dir)

In [26]:
# variant_list = ['RP2_Cys86Tyr', 'RP2_Leu253Arg', 'HBD_Gly26Asp', 'EFHC1_Asp253Tyr',
#        'PKP2_Ala526Thr', 'PMP22_Asp37Val', 'SMAD1_Val3Ala',
#        'DCX_Ser129Leu', 'TTPA_Gly246Arg', 'FXN_Trp173Gly',
#        'EFHC1_Cys259Tyr', 'IL2RG_Tyr89Cys', 'SFTPC_Ile73Thr',
#        'PLP1_Gly186Cys', 'ASNS_Ala6Glu', 'EFHC1_Met448Thr',
#        'SH3BP2_Gly420Glu', 'KCTD7_Asn273Ile', 'EIF2B4_Ala228Val',
#        'ACSF3_Ser431Tyr', 'SFTPC_Leu188Gln', 'SMN2_Gly287Arg',
#        'POMT1_Trp560Cys', 'SFTPC_Leu194Pro', 'PLP1_Ile152Thr',
#        'EFHC1_Asp210Asn', 'FBP1_Phe194Ser', 'TAT_Arg433Trp',
#        'DES_Ala337Pro', 'COMP_Asn523Lys', 'AIPL1_Met79Thr',
#        'PLP1_Ser135Phe', 'DIABLO_Ser126Leu', 'AGXT_Ile202Asn']

# variant_list_2 = ['ACSF3_Met266Val',
#  'ACSF3_Gly119Asp',
#  'ACSF3_Arg558Trp',
#  'ACSF3_Arg471Trp',
#  'ACSF3_Pro243Leu',
#  'ACSF3_Asp457Asn',
#  'ACSF3_Thr358Ile']

variant_list_3 = ['PLA2G6_Thr585Ile',
            'PLA2G6_Leu481Gln',
            'PLA2G6_Arg600Gln',
            'PLA2G6_Val310Glu',
            'PLA2G6_Glu567Lys',
            'PLA2G6_Arg538Cys']

# variant_list += variant_list_2 + variant_list_3

for variant in tqdm(variant_list_3):
    plot_allele_cell(meta_data, variant, "GFP", auroc_df=local_wtvar, site="05", max_intensity=0.99, display=False, imgs_dir=tiff_imgs_dir, output_dir=out_cells_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Image for PLA2G6_Thr585Ile already exists.
5 PLA2G6_Thr585Ile N13
5 PLA2G6 D13


 17%|█▋        | 1/6 [00:04<00:20,  4.19s/it]

Image for PLA2G6_Leu481Gln already exists.
3 PLA2G6_Leu481Gln P13
5 PLA2G6_Leu481Gln P13
7 PLA2G6_Leu481Gln P13
5 PLA2G6 D13


 33%|███▎      | 2/6 [00:07<00:15,  3.88s/it]

...

 67%|██████▋   | 4/6 [00:15<00:07,  3.85s/it]

Image for PLA2G6_Glu567Lys already exists.
5 PLA2G6_Glu567Lys F13
7 PLA2G6_Glu567Lys F13
5 PLA2G6 D13


 83%|████████▎ | 5/6 [00:19<00:03,  3.87s/it]

Image for PLA2G6_Arg538Cys already exists.
3 PLA2G6_Arg538Cys H13
5 PLA2G6_Arg538Cys H13
5 PLA2G6 D13


100%|██████████| 6/6 [00:23<00:00,  3.89s/it]
